In [7]:
#Project No. 1: Mercedes-Benz Greener Manufacturing

In [8]:
# Step1: Import the required libraries

In [9]:
# linear algebra

In [10]:
import numpy as np

In [11]:
import pandas as pd

In [12]:
from sklearn.decomposition import PCA

In [13]:
# Step2: Read the data from train.csv

In [14]:
df_train = pd.read_csv('train.csv')

In [15]:
print('Size of training set: {} rows and {} columns'
      .format(*df_train.shape))

Size of training set: 4209 rows and 378 columns


In [16]:
df_train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#Step3: Collect the Y values into an array

In [18]:
y_train = df_train['y'].values

In [19]:
# Step4: Understand the data types we have

In [20]:
cols = [c for c in df_train.columns if 'X' in c]

In [21]:
print('Number of features: {}'.format(len(cols)))

Number of features: 376


In [22]:
print('Feature types:')
df_train[cols].dtypes.value_counts()

Feature types:


int64     368
object      8
dtype: int64

In [23]:
# Step5: Count the data in each of the columns

In [24]:
counts = [[], [], []]
for c in cols:
    typ = df_train[c].dtype
    uniq = len(np.unique(df_train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)

In [25]:
print('Constant features: {} Binary features: {} Categorical features: {}\n'
      .format(*[len(c) for c in counts]))

Constant features: 12 Binary features: 356 Categorical features: 8



In [26]:
print('Constant features:', counts[0])

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']


In [27]:
print('Categorical features:', counts[2])

Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [28]:
# Step6: Read the test.csv data

In [29]:
df_test = pd.read_csv('test.csv')

In [30]:
# remove columns ID and Y from the data as they are not used for learning

In [31]:
usable_columns = list(set(df_train.columns) - set(['ID', 'y']))
y_train = df_train['y'].values
id_test = df_test['ID'].values

x_train = df_train[usable_columns]
x_test = df_test[usable_columns]

In [32]:
# Step7: Check for null and unique values for test and train sets

In [33]:
def check_missing_values(df):
    if df.isnull().any().any():
        print("There are missing values in the dataframe")
    else:
        print("There are no missing values in the dataframe")

In [34]:
check_missing_values(x_train)
check_missing_values(x_test)

There are no missing values in the dataframe
There are no missing values in the dataframe


In [35]:
# Step8: If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
# Apply label encoder

In [36]:
for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one 
        # value is useless so we drop it
        x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [37]:
x_train.head()

,X359,X295,X305,X357,X270,X330,X257,X233,X15,X164,...,X45,X360,X170,X262,X302,X347,X38,X182,X59,X225
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# Step9: Make sure the data is now changed into numericals

In [39]:
print('Feature types:')
x_train[cols].dtypes.value_counts()

Feature types:


int64    376
dtype: int64

In [40]:
# Step10: Perform dimensionality reduction
# Linear dimensionality reduction using Singular Value Decomposition of 
# the data to project it to a lower dimensional space.

In [41]:
n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

In [42]:
# Step11: Training using xgboost

In [43]:
import xgboost as xgb

In [44]:
from sklearn.metrics import r2_score

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
x_train, x_valid, y_train, y_valid = train_test_split(pca2_results_train, y_train, test_size=0.2, random_state=4242)

In [47]:
d_train = xgb.DMatrix(x_train, label=y_train)


In [48]:
d_valid = xgb.DMatrix(x_valid, label=y_valid)

In [49]:
d_test = xgb.DMatrix(pca2_results_test)

In [50]:
params = {}

In [51]:
params['objective'] = 'reg:linear'

In [52]:
params['eta'] = 0.02

In [53]:
params['max_depth'] = 4

In [54]:
def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

In [55]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [56]:
clf = xgb.train(params, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

[13:08:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:99.1484	valid-rmse:98.263	train-r2:-58.353	valid-r2:-67.6375
Multiple eval metrics have been passed: 'valid-r2' will be used for early stopping.

Will train until valid-r2 hasn't improved in 50 rounds.
[10]	train-rmse:81.2765	valid-rmse:80.3643	train-r2:-38.8843	valid-r2:-44.9101
[20]	train-rmse:66.7161	valid-rmse:65.7733	train-r2:-25.874	valid-r2:-29.7526
[30]	train-rmse:54.8696	valid-rmse:53.8896	train-r2:-17.1775	valid-r2:-19.6439
[40]	train-rmse:45.2449	valid-rmse:44.22	train-r2:-11.3598	valid-r2:-12.9001
[50]	train-rmse:37.4474	valid-rmse:36.3724	train-r2:-7.46669	valid-r2:-8.40431
[60]	train-rmse:31.1476	valid-rmse:30.0188	train-r2:-4.85761	valid-r2:-5.40575
[70]	train-rmse:26.0868	valid-rmse:24.9091	train-r2:-3.10878	valid-r2:-3.41059
[80]	train-rmse:22.0466	valid-rmse:20.8327	train-r2:-1.93465	valid-r2:-2.08513
[90]	train-rmse:18.8444	valid-rmse:17.

In [57]:
# Step12: Predict your test_df values using xgboost

In [58]:
p_test = clf.predict(d_test)

In [59]:
sub = pd.DataFrame()

In [60]:
sub['ID'] = id_test

In [61]:
sub['y'] = p_test

In [62]:
sub.to_csv('xgb.csv', index=False)

In [63]:
sub.head()

,ID,y
0,1,82.661789
1,2,97.046974
2,3,82.948067
3,4,76.719528
4,5,112.973297
